In [6]:
import os
import pydot
import numpy as np

from pydrake.geometry import StartMeshcat
from pydrake.systems.analysis import Simulator
from IPython.display import HTML, SVG, display
from pydrake.visualization import ModelVisualizer
from pydrake.all import (
  ConstantVectorSource, 
  PiecewisePose,  
  Simulator, 
  StartMeshcat, 
  DiagramBuilder, 
  MeshcatVisualizer, 
  namedview,
  RigidTransform, 
  RollPitchYaw, RotationMatrix, LeafSystem, AbstractValue,
)
from manipulation import running_as_notebook
from manipulation.station import MakeHardwareStation, load_scenario
from manipulation.meshcat_utils import AddMeshcatTriad
from manipulation.scenarios import AddIiwaDifferentialIK
# from pydrake.systems.System import CreateDefaultContext

from constants import *

In [7]:
# Start the visualizer.
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at http://localhost:7001


In [8]:
def make_stack():
    output = ''
    for i in range(NUM_LAYERS):
        # alternate block orientations
        layer_in_x_direction = i % 2 == 0
        for j in range(NUM_BLOCKS_PER_LAYER):
            x = (BLOCK_WIDTH * (j - 1)) if layer_in_x_direction else 0
            y = 0 if layer_in_x_direction else (BLOCK_WIDTH * (j - 1))
            z = BLOCK_HEIGHT * (i + 0.5) + TABLE_HEIGHT
            orientation = "{ deg: [0, 0, 0] }" if layer_in_x_direction else "{ deg: [0, 0, 90] }"
            output += f"""
- add_model:
    name: block_{i * 3 + j}
    file: package://jengabot/models/jenga_block.sdf
    default_free_body_pose:
        jenga_block_link:
            translation: [{x}, {y}, {z}]
            rotation: !Rpy {orientation}"""
    return output

In [9]:
class PoseTrajectorySource(LeafSystem):
    def __init__(self, pose_trajectory):
        LeafSystem.__init__(self)
        self._pose_trajectory = pose_trajectory
        self.DeclareAbstractOutputPort(
            "pose", lambda: AbstractValue.Make(RigidTransform()), self.CalcPose
        )

    def CalcPose(self, context, output):
        output.set_value(self._pose_trajectory.GetPose(context.get_time()))

In [10]:
def push_motion(block, num_keyframes):
    assert num_keyframes >= 2, "There must be at least two frames for this motion"

    layer, index = block
    layer_in_x_direction = layer % 2 == 0
    # hit center of the block's smaller face from the side closer to the Iiwa
    if layer_in_x_direction:
        target_x = BLOCK_WIDTH * (index - 1)
        target_y = BLOCK_LENGTH / 2
    else:
        target_x = BLOCK_LENGTH / 2
        target_y = BLOCK_WIDTH * (index - 1)
    target_z = BLOCK_HEIGHT * (layer + 0.5) + TABLE_HEIGHT
    start_position = np.array([target_x, target_y, target_z])

    # orient the gripper such that it will push out the block if it moves forward
    yaw = np.pi if layer_in_x_direction else np.pi * 3 / 2
    target_rotation = RollPitchYaw(0, 0, yaw).ToRotationMatrix()

    # construct a trajectory that pushes the block out part of the way
    shift_direction = np.array([0, 1, 0]) if layer_in_x_direction else np.array([1, 0, 0])
    shift = shift_direction * PUSH_FRACTION * BLOCK_LENGTH
    frac = lambda i: i / (num_keyframes - 1)
    return [RigidTransform(target_rotation, start_position + shift * frac(i)) for i in range(num_keyframes)]

In [15]:
def construct_scenario():
    scenario_data = """
directives:
- add_model:
    name: table_top
    file: package://jengabot/models/table_top.sdf
- add_weld:
    parent: world
    child: table_top::table_top_center
- add_model:
    name: iiwa
    file: package://drake/manipulation/models/iiwa_description/iiwa7/iiwa7_no_collision.sdf
    default_joint_positions:
        iiwa_joint_1: [-1.57]
        iiwa_joint_2: [0.1]
        iiwa_joint_3: [0]
        iiwa_joint_4: [-1.2]
        iiwa_joint_5: [0]
        iiwa_joint_6: [ 1.6]
        iiwa_joint_7: [0]
- add_weld:
    parent: table_top_link
    child: iiwa::iiwa_link_0
    X_PC:
        translation: [0.35, 0.5, 0.015]
- add_model:
    name: wsg
    file: package://drake/manipulation/models/wsg_50_description/sdf/schunk_wsg_50_with_tip.sdf
- add_weld:
    parent: iiwa::iiwa_link_7
    child: wsg::body
    X_PC:
        translation: [0, 0, 0.09]
        rotation: !Rpy { deg: [90, 0, 90]}
"""

    driver_data = """
model_drivers:
    iiwa: !IiwaDriver
      hand_model_name: wsg
    wsg: !SchunkWsgDriver {}
"""

    scenario_data += make_stack()
    scenario_data += driver_data
    return load_scenario(data = scenario_data)

In [12]:
def construct_trajectory():
    total_time = 20
    num_keyframes = 10
    times = np.linspace(0, total_time, num_keyframes)
    keyframes = push_motion((3, 0), num_keyframes)
    return PiecewisePose.MakeLinear(times, keyframes)

In [ ]:
scenario = construct_scenario()
builder = DiagramBuilder()

station = builder.AddSystem(MakeHardwareStation(scenario, package_xmls=['./package.xml']))
plant = station.GetSubsystemByName("plant")
controller_plant = station.GetSubsystemByName("iiwa.controller").get_multibody_plant_for_control()

visualizer = MeshcatVisualizer.AddToBuilder(builder, station.GetOutputPort("query_object"), meshcat)

traj_source = builder.AddSystem(PoseTrajectorySource(construct_trajectory()))
controller = AddIiwaDifferentialIK(builder, controller_plant, frame=controller_plant.GetFrameByName("body"))
builder.Connect(traj_source.get_output_port(), controller.get_input_port(0))
builder.Connect(station.GetOutputPort("iiwa.state_estimated"), controller.GetInputPort("robot_state"),)
builder.Connect(controller.get_output_port(), station.GetInputPort("iiwa.position"))

wsg_position = builder.AddSystem(ConstantVectorSource([0]))
builder.Connect(wsg_position.get_output_port(), station.GetInputPort("wsg.position"),)

diagram = builder.Build()

simulator = Simulator(diagram)
context = simulator.get_mutable_context()
station_context = station.GetMyMutableContextFromRoot(context)
plant_context = plant.GetMyMutableContextFromRoot(context)

q0 = plant.GetPositions(plant_context, plant.GetModelInstanceByName("iiwa"))
station.GetInputPort("iiwa.position").FixValue(station_context, q0)

station.GetInputPort("wsg.position").FixValue(station_context, [0])

simulator.AdvanceTo(0.1)

INFO:drake:Meshcat listening for connections at http://localhost:7005
==== LCM Warning ===
LCM detected that large packets are being received, but the kernel UDP
receive buffer is very small.  The possibility of dropping packets due to
insufficient buffer space is very high.

For more information, visit:
   http://lcm-proj.github.io/lcm/multicast_setup.html

==== LCM Warning ===
LCM detected that large packets are being received, but the kernel UDP
receive buffer is very small.  The possibility of dropping packets due to
insufficient buffer space is very high.

For more information, visit:
   http://lcm-proj.github.io/lcm/multicast_setup.html

==== LCM Warning ===
LCM detected that large packets are being received, but the kernel UDP
receive buffer is very small.  The possibility of dropping packets due to
insufficient buffer space is very high.

For more information, visit:
   http://lcm-proj.github.io/lcm/multicast_setup.html

==== LCM Warning ===
LCM detected that large packets are 

In [ ]:
# pick central target
# push block forward or back
# grab firmly and pull out
# move without knocking over stack
# place on top of stack

In [ ]:


def move(self, sim_duration=20.0):
    context = CreateDefaultContext()
    simulator = Simulator(diagram, context)
    simulator.set_target_realtime_rate(1.0)

    duration = sim_duration if running_as_notebook else 0.01
    simulator.AdvanceTo(duration)